# EDA & Investigate Data

- Use PySpark, Pandas and the Spark NLP Lib

<BR>

**Dataset Descriptions**

| NAME | SHAPE | DESCRIPTION |
| ---- | ----------- | ---- | 
| Posts.csv | (6947, 11) |A collection of all posts scraped from Red|
| Comments.csv | (963817, 5) |A collection of all posts|

<BR>

<BR>

<BR>

Dependencies:
- pip install spark-nlp==3.3.3 (for NLP Package)

## Reddit Data NLP-ification

In [2]:
# Import Spark NLP & Start SparkSession
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline

from pyspark.sql import SparkSession

from pyspark.ml import Pipeline # Start Spark Session with Spark NLP


# Instantiate SparkNLP Session
spark = sparknlp.start()
spark = SparkSession.builder \
    .appName("Reddit Posts Study") \
    .config('spark.jars.packages', #need specify this spark session to read in data (and do PySpark df stuff)
            'com.johnsnowlabs.nlp:spark-nlp_2.11:2.3.5') \
    .getOrCreate()

In [31]:
spark

In [3]:
# Read in Data & Convert to Spark (easier than reading sep)
import pandas as pd 
from nltk.corpus import stopwords
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, StringType, FloatType, StructType, StructField

# Read in NLP Data
posts = pd.read_csv("/Volumes/GoogleDrive/My Drive/Colab/IDS 561 - Big Data/Data/Reddit/Concated/Posts.csv")
comments = pd.read_csv("/Volumes/GoogleDrive/My Drive/Colab/IDS 561 - Big Data/Data/Reddit/Concated/Comments.csv")

#print(posts.shape, "\n", posts.columns)
#print(comments.shape, "\n", comments.columns)


# Set Stopwords for English
try :
    eng_stopwords = stopwords.words('english')
except:
    nltk.download('stopwords')
    eng_stopwords = stopwords.words('english')


#eng_stopwords.append('xxxx') #method for adding an additional stopword 


# Create Schemas for Dataframes
posts_schema = StructType([
 StructField("title", StringType(), True), 
 StructField("score", IntegerType(), True), 
 StructField("id", StringType(), True), 
 StructField("subreddit_name", StringType(), True), 
 StructField("url", StringType(), True), 
 StructField("number_comments", IntegerType(), True), 
 StructField("ups", IntegerType(), True), 
 StructField("upvote_ratio", FloatType(), True), 
 StructField("date_created", FloatType(), True), #will convert to datetime in df, for schema will settle for float & not `TimestampType`
 StructField("body", StringType(), True), 
 StructField("keyword", StringType(), True)         ])

comments_schema = StructType([
 StructField("comment_id", StringType(), True), 
 StructField("parent_id", StringType(), True), 
 StructField("comment_body", StringType(), True), 
 StructField("comment_link_id", StringType(), True), 
 StructField("keyword", StringType(), True)         ])

# Send dfs to PySpark & Add datetime
spark_posts = spark.createDataFrame(posts, posts_schema) 
spark_posts = spark_posts.withColumn("datetime", from_unixtime(col("date_created"), "yyyy-MM-dd")) #add datetime to df
#spark_posts.printSchema()
spark_posts.show()

spark_comments = spark.createDataFrame(comments, comments_schema) 
spark_comments = spark_comments.withColumnRenamed("comment_body", "body")
#spark_comments.printSchema()
spark_comments.show()

+--------------------+-----+------+--------------+--------------------+---------------+---+------------+------------+--------------------+-------+----------+
|               title|score|    id|subreddit_name|                 url|number_comments|ups|upvote_ratio|date_created|                body|keyword|  datetime|
+--------------------+-----+------+--------------+--------------------+---------------+---+------------+------------+--------------------+-------+----------+
|        To the moon!|  484|mqqcbt|         r/btc|https://i.redd.it...|            131|484|        0.93|1.61840614E9|                 NaN|   moon|2021-04-14|
|/r/bitcoin is in ...|  486|7pvojj|         r/btc|https://www.reddi...|            168|486|        0.84|1.51575181E9|Technological inf...|   moon|2018-01-12|
|/r/bitcoin for ye...|  470|7lcafe|         r/btc|https://www.reddi...|             72|470|        0.85|1.51388902E9|The Bitcoin subre...|   moon|2017-12-21|
|This Excitement a...|  403|8elr2k|         r/btc|ht

In [ ]:
comments.info()

In [4]:
# Experimenting w Adding DateTime col
from pyspark.sql.functions import *
#sp2 = spark_posts.select(from_unixtime(col("date_created"), "yyyy-MM-dd").alias("datetime")) #create col in sep df 
spark_posts = spark_posts.withColumn("datetime", from_unixtime(col("date_created"), "yyyy-MM-dd")) #add datetime to df
spark_posts.show()

+--------------------+-----+------+--------------+--------------------+---------------+---+------------+------------+--------------------+-------+----------+
|               title|score|    id|subreddit_name|                 url|number_comments|ups|upvote_ratio|date_created|                body|keyword|  datetime|
+--------------------+-----+------+--------------+--------------------+---------------+---+------------+------------+--------------------+-------+----------+
|        To the moon!|  484|mqqcbt|         r/btc|https://i.redd.it...|            131|484|        0.93|1.61840614E9|                 NaN|   moon|2021-04-14|
|/r/bitcoin is in ...|  486|7pvojj|         r/btc|https://www.reddi...|            168|486|        0.84|1.51575181E9|Technological inf...|   moon|2018-01-12|
|/r/bitcoin for ye...|  470|7lcafe|         r/btc|https://www.reddi...|             72|470|        0.85|1.51388902E9|The Bitcoin subre...|   moon|2017-12-21|
|This Excitement a...|  403|8elr2k|         r/btc|ht

#### Clean Up Datasets & Compute DownVotes

In [8]:
# Deprecated
posts["id"]

comments["parent_id"] = comments["parent_id"].str.slice(start=3) #get rid of non-useful t3 attribute
comments["comment_link_id"] = comments["comment_link_id"].str.slice(start=3) #comment_link_ids are diff than parent! They refer to whom this comment is replying! 

In [31]:
posts

,title,score,id,subreddit_name,url,number_comments,ups,upvote_ratio,date_created,body,keyword
0,To the moon!,484,mqqcbt,r/btc,https://i.redd.it/mpgx359u25t61.jpg,131,484,0.93,1.618406e+09,NaN,moon
1,/r/bitcoin is in uproar about Coinbase not imp...,486,7pvojj,r/btc,https://www.reddit.com/r/btc/comments/7pvojj/r...,168,486,0.84,1.515752e+09,Technological inferiority when bitcoin grinds ...,moon
2,"/r/bitcoin for years: No altcoin discussion, h...",470,7lcafe,r/btc,https://www.reddit.com/r/btc/comments/7lcafe/r...,72,470,0.85,1.513889e+09,The Bitcoin subreddit reaches new levels of hy...,moon
3,This Excitement and Energy is what Bitcoin fel...,403,8elr2k,r/btc,https://www.reddit.com/r/btc/comments/8elr2k/t...,215,403,0.78,1.524587e+09,"For anyone that wasn't around in 2013, THIS is...",moon
4,MOON!,300,bqelqr,r/btc,https://i.redd.it/z7d7gbg2r4z21.jpg,69,300,0.89,1.558255e+09,NaN,moon
...,...,...,...,...,...,...,...,...,...,...,...
6942,John Carvalho and Roger Ver on Bitcoin Cash vs...,376,7fvpag,r/Bitcoin,https://www.youtube.com/watch?v=OJT2CbfHTpo&fe...,199,376,0.86,1.511795e+09,NaN,cash
6943,Exposed: How Bankers are trying to centralize ...,378,74wejs,r/Bitcoin,https://www.reddit.com/r/Bitcoin/comments/74we...,87,378,0.81,1.507399e+09,^^*Open ^^Bazaar ^^was ^^crossed-out ^^after ^...,cash
6944,Donald Trump has been bashing Bitcoin becuase ...,379,pg87gt,r/Bitcoin,https://www.reddit.com/r/Bitcoin/comments/pg87...,261,379,0.77,1.630548e+09,Trump scared. Bitcoin will replace real estate...,cash
6945,We are developing an all in one payment card t...,376,1ssrfx,r/Bitcoin,https://www.reddit.com/r/Bitcoin/comments/1ssr...,184,376,0.94,1.386947e+09,Just like the title says we are developing a d...,cash


In [35]:
# Compute Total Num Votes (could easily get downvotes)
upvotes = 484

percentage_missing = 1 - .93
total_votes = (1+percentage_missing) * 484
missing_votes = total_votes - upvotes
484/517.87
missing_votes

33.87999999999988

In [30]:
comments

,comment_id,parent_id,comment_body,comment_link_id,keyword
0,guhvb5q,mqqcbt,Dude this is awesome!,mqqcbt,moon
1,guhwfda,mqqcbt,Great work!,mqqcbt,moon
2,gui8buh,mqqcbt,I have [the original design](https://www.behan...,mqqcbt,moon
3,guicy3c,mqqcbt,To the moon!!! ┗(°0°)┛,mqqcbt,moon
4,guhypo5,mqqcbt,To the Mars!,mqqcbt,moon
...,...,...,...,...,...
963812,cg3wz9k,cg3ppfp,"If you plan on hedging, why take the bet at al...",20gkdx,cash
963813,cg3fdpc,cg3f0q9,"I agree with much of this, prudence is warrant...",20gkdx,cash
963814,cg3g4xw,cg3f0q9,"If there were, say, a 100 million bitcoins the...",20gkdx,cash
963815,cg3fgjx,cg3fdpc,>But wanted to point out that one of Bitcoin's...,20gkdx,cash


#### Build NLP Pipeline

1. Apply to Posts df first (then to comments as secondary)
2. May want to aggregate the comments df and append to post, Potential Features that we can "extend" the posts df with
    - total_num_comments
    - num_positive_comments
    - average_pos_score
    - average_neg_score
    - topics or main keywords/ideas present in the comments (related to that post)
    - Is a meme? (does the post contain comical stuff or a image?)


<BR>

`Reference Articles`
- NLP in PySpark: https://towardsdatascience.com/natural-language-processing-with-pyspark-and-spark-nlp-b5b29f8faba
- LDA Categorizer: https://www.analyticsvidhya.com/blog/2020/07/build-text-categorization-model-with-spark-nlp/

In [30]:
# NLP Pipeline
from pyspark.ml import Pipeline
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer, LemmatizerModel, StopWordsCleaner, SentimentDetector)


# Instantiate Annotators - (https://nlp.johnsnowlabs.com/docs/en/annotators)
documentAssembler = DocumentAssembler().setInputCol('body').setOutputCol('document')

tokenizer = Tokenizer().setInputCols(['document']).setOutputCol('token')

normalizer = Normalizer().setInputCols(['token']).setOutputCol('normalized').setLowercase(False)
# Normalizer defaults to lowercasing all words, setLowercase = True to maintain casing
#lots of BS in the reddit posts

# note that lemmatizer needs a dictionary. So I used the pre-trained
# model (note that it defaults to english)
lemmatizer = LemmatizerModel.pretrained().setInputCols(['normalized']).setOutputCol('lemma')

stopwords_cleaner = StopWordsCleaner().setInputCols(['lemma']).setOutputCol('clean_lemma').setCaseSensitive(False).setStopWords(eng_stopwords)

# sentimentDetector Anaylzes a dict file of words for sentiment
sentimentDetector = SentimentDetector() \
    .setInputCols(["lemma", "document"]) \
    .setOutputCol("sentimentScore") \
    .setDictionary("sentiment-dict.txt", ",", ReadAs.TEXT)
    #.setEnableScore(True)\

# finisher converts tokens to human-readable output
finisher = Finisher().setInputCols(['clean_lemma']).setCleanAnnotations(False)


# Assemble the Annotators
pipeline = Pipeline() \
     .setStages([
           documentAssembler,
           tokenizer,
           normalizer,
           lemmatizer,
           stopwords_cleaner,
           sentimentDetector,
           finisher
     ])


# Apply Pipeline to Reddit dfs
posts_nlp = pipeline.fit(spark_posts).transform(spark_posts)
comments_nlp = pipeline.fit(spark_comments).transform(spark_comments)

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [ ]:
# View Output from Above
#posts_nlp.selectExpr("sentimentScore").show(truncate=False)
posts_nlp.select("sentimentScore.result").distinct().show()


# Words for Sentiment Classifier

# Optimistic Words
cheering,positive
comforting,positive
reassuring,positive
soothing,positive
assured,positive
confident,positive
decisive,positive
whale,positive
doubtless,positive
positive,positive
sure,positive
unhesitating,positive
beamish,positive
bullish,positive
bull,positive
eupeptic,positive
favorable,positive
good,positive
auspicious,positive
bright,positive
encouraging,positive
fair,positive
golden,positive
heartening,positive
hopeful,positive
likely,positive
promising,positive
propitious,positive
rose-colored,positive
roseate,positive
rosy,positive
upbeat,positive
ath,positive
gains,positive
anarcho-capitalism,positive
anarcho,positive
anti-fragile,positive
bitcoiner,positive
anti-malware,positive
aml,positive
diamond,positive
inflation,positive
resistance,positive
bagholder,positive
fomo,positive
kimchi,positive
anitvirus,positive
laser,positive
long,positive
oversold,positive



# Pessimistic Words
cynical,negative
fatalistic,negative
nihilist,negative
nihilistic,negative
desperate,negative
discouraging,negative
disheartening,negative
inauspicious,negative
unlikely,negative
unpromising,negative
bleak,negative
cheerless,negative
comfortless,negative
depressing,negative
desolate,negative
dismal,negative
dreary,negative
funereal,negative
gloomy,negative
morose,negative
saturnine,negative
sepulchral,negative
somber,negative
sullen,negative
grim,negative
fish ,negative
deflation,negative
contrary,negative
hostile,negative
negative,negative
bearish,negative
defeatist,negative
despairing,negative
downbeat,negative
hopeless,negative
hacking,negative
hard,negative
jomo,negative
fork,negative
censorship,negative
overbought,negative
scamcoin,negative
weak,negative
exit,negative
minnow,negative
newb,negative
no-coiner,negative
rug,negative
scam,negative
dead,negative
memecoin,negative
bear,negative
phishing,negative
ponzi,negative
pyramid,negative
dump,negative
correction,negative
bots,negative
bearwhale,negative
atl ,negative
astroturfing,negative
trap,negative
bfa,negative

In [10]:
# Example of Filtering df in PySpark
#posts_nlp.printSchema()
posts_nlp.show()
comments_nlp.show()
#one_post = posts_nlp.filter(posts_nlp.id == "7pvojj")
#one_post.select(["body", "finished_clean_lemma"]).show(truncate=False)

+--------------------+-----+------+--------------+--------------------+---------------+---+------------+------------+--------------------+-------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|score|    id|subreddit_name|                 url|number_comments|ups|upvote_ratio|date_created|                body|keyword|  datetime|            document|               token|          normalized|               lemma|         clean_lemma|finished_clean_lemma|
+--------------------+-----+------+--------------+--------------------+---------------+---+------------+------------+--------------------+-------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|        To the moon!|  484|mqqcbt|         r/btc|https://i.redd.it...|            131|484|        0.93|1.61840614E9|                 NaN|   moon|20

In [11]:
# Select all Unique Values in a Column
posts_nlp.select("subreddit_name").distinct().collect()

[Row(subreddit_name='r/btc'), Row(subreddit_name='r/Bitcoin')]

In [ ]:
# Send Posts to a PandasFrame + Verify Date Range
posts_nlp.show()
p_nlp = posts_nlp.toPandas()

print("min-", p_nlp["datetime"].min(), "to", p_nlp["datetime"].max(),"-max") #verified date range

In [82]:
# Drop some useless columns 

p_cols = ("date_created", "lemma", "normalized", "token", "clean_lemma", "document")
c_cols = ("comment_link_id", "lemma", "normalized", "token", "clean_lemma", "document")
# Don't need these NLP Pipe cols- ["lemma", "normalized", "token"] as they are intermediate steps
# (can be recomputed with NLP Pipe if need be)

posts_nlp = posts_nlp.drop(*p_cols)
comments_nlp.drop(*c_cols)

DataFrame[comment_id: string, parent_id: string, body: string, keyword: string, sentimentScore: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, finished_clean_lemma: array<string>]

In [ ]:
# Ideas for Combining `Comments` & `Posts`

'''
Create new features in posts based on the data in Comments & Join on Parent IDs

Potential Features to Create:
1. Number of Unique Users in Comments (as proportion of total users?)
2. Total Number of Comments (as a volume metric or measure of "Hype" around a post's topic)
3. Optimism & Pessimism (for all of the comments relating to a post, do the comments follow a certain type of trend that we can quantify)
  this notion of "Aggregate Semantics" is also interesting & could be a point of interest for further research (particularly after seeing the WSB's group make waves)
  These features & other "Semantic Measures" could be used to create the radar charts! (Awesome way to visually compare the meaning of different posts?)
4. Total Number of Languages Present in Each Post
  Most popular Languages overall? 
'''

In [40]:
# Testing Vader for CommentWise Sentiment Analysis -- Added to Below!
import pyspark.sql.functions as F
import pyspark.sql.types as T
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#comments_nlp.show()

sentimentifier = F.udf(lambda x: list(SentimentIntensityAnalyzer().polarity_scores(x).values())[-1])
comments_nlp = comments_nlp.withColumn('sentiment_score', sentimentifier('finished_clean_lemma')) #`body` was original (both work with very little difference in scores -- order of .00x change)

In [ ]:
comments_nlp_test.select("sentiment_score").show()
#comments_nlp.select("finished_clean_lemma").show()
#comments_nlp.select("body").show()

'''
Head of Sentiment Score Calculated with `Body`
  "finished_clean_lemma"          "body"
    +---------------+       +---------------+
    |sentiment_score|       |sentiment_score|
    +---------------+       +---------------+
    |         0.6588|       |         0.6588|
    |         0.6588|       |         0.6588|
    |         0.3802|       |         0.3802|
    |            0.0|       |            0.0|
    |            0.0|       |            0.0|
    |            0.0|       |            0.0|
    |            0.0|       |            0.0|
    |         0.2263|       |         0.2263|
    |         0.9453|       |         0.9453|
    |         0.6124|       |         0.6124|
    |         0.4588|       |         0.4588|
    |         0.8805|       |         0.8805|
    |         0.9421|       |         0.9421|
    |         0.8176|       |         0.8176|
    |         0.9779|       |         0.9779|
    |         0.5684|       |         0.5684|
    |         0.7078|       |         0.7078|
    |         -0.567|       |         -0.567|
    |        -0.2732|       |        -0.2732|
    |            0.0|       |            0.0|
    +---------------+       +---------------+
*both of above were the sentiment scores after calculating values based on either the cleaned lemma or base body text

'''

In [36]:
# Vader Sentiment Func Test
a = "This is good"
SentimentIntensityAnalyzer().polarity_scores(a).values()
vals = list(SentimentIntensityAnalyzer().polarity_scores(a).values())
vals[-1] #position of compound sentiment value (combined sentiment of all tokens in sentence: [pos, neg, neutral])

dict_values([0.0, 0.408, 0.592, 0.4404])

In [52]:
len(comments_nlp.select("comment_id").distinct().collect()) #comment_link_id is what relates to parent post!

690703

In [49]:
len(posts_nlp.select("id").distinct().collect())

5685

In [50]:
comments_nlp.show()

+----------+---------+--------------------+---------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+
|comment_id|parent_id|                body|comment_link_id|keyword|            document|               token|          normalized|               lemma|         clean_lemma|finished_clean_lemma|sentiment_score|
+----------+---------+--------------------+---------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+
|   guhvb5q|   mqqcbt|Dude this is awes...|         mqqcbt|   moon|[{document, 0, 20...|[{token, 0, 3, Du...|[{token, 0, 3, Du...|[{token, 0, 3, Du...|[{token, 0, 3, Du...|     [Dude, awesome]|            0.0|
|   guhwfda|   mqqcbt|         Great work!|         mqqcbt|   moon|[{document, 0, 10...|[{token, 0, 4, Gr...|[{token, 0, 4, Gr...|[{token, 0, 4, Gr...|[{token, 

In [44]:
# Get Count of Comments Per Post
posts_nlp.select("id").show()

comments_nlp.select("id").show()

+------+
|    id|
+------+
|mqqcbt|
|7pvojj|
|7lcafe|
|8elr2k|
|bqelqr|
|8ezs3p|
|7bmux7|
|6nbv53|
|7w43mb|
|58u8em|
|4klqtg|
|7c1wtm|
|5vbofp|
|mbsdqv|
|mswhzl|
|7f1uzq|
|77p2my|
|71i6y3|
|91bjvu|
|6ubr5l|
+------+
only showing top 20 rows



In [76]:
import pyspark.sql.functions as func

post_comment_counts = comments_nlp.groupBy(["comment_link_id", "keyword"]).count().orderBy("comment_link_id")

# Get Count of Uniques
#post_comment_uniques = comments_nlp.groupBy(["comment_link_id","comment_id"]).sum().orderBy("comment_link_id")
post_comment_uniques = comments_nlp.groupby("comment_link_id", "keyword").\
    agg(
        func.countDistinct("comment_id").alias("unique_users")
        
    )

In [57]:
post_comment_counts.show() #6,777 of these 
#post_comment_counts.write.csv("Counts-PostComments.csv", singlefile=True)
#post_comment_counts.coalesce(1).write.format("csv").mode("overwrite").save("Counts-PostComments.csv")

# Write out Single CSV - works!
#pcc = post_comment_counts.toPandas()
#pcc.to_csv("Counts-PostComments.csv")

+---------------+--------------+-----+
|comment_link_id|       keyword|count|
+---------------+--------------+-----+
|         171chv|          wars|   57|
|         18jad6|cryptocurrency|  160|
|         1a4ab0|          fork|  287|
|         1a4ab0|        blocks|  287|
|         1a5b3b|     blocksize|   78|
|         1a65u9|          fork|   36|
|         1admpi|   centralised|   68|
|         1alflg|   centralised|    4|
|         1bbo23|        energy|   54|
|         1bn1yx|        mining|  154|
|         1bn1yx|cryptocurrency|  154|
|         1bn1yx|          fiat|  154|
|         1bn1yx|          cash|  154|
|         1bnm3i|        mining|  266|
|         1bnm3i|          fork|  266|
|         1bxcm4|        mining|  275|
|         1bxrrw|    currencies|  347|
|         1bxrrw|          fiat|  347|
|         1bxrrw|        blocks|  347|
|         1bxrrw|        mining|  347|
+---------------+--------------+-----+
only showing top 20 rows



In [78]:
#post_comment_uniques.show()

# Write out Single CSV 
pcu = post_comment_uniques.toPandas()
pcu.to_csv("Uniques-PostComments.csv")

In [83]:
posts_nlp.show()

+--------------------+-----+------+--------------+--------------------+---------------+---+------------+--------------------+-------+----------+--------------------+--------------------+
|               title|score|    id|subreddit_name|                 url|number_comments|ups|upvote_ratio|                body|keyword|  datetime|      sentimentScore|finished_clean_lemma|
+--------------------+-----+------+--------------+--------------------+---------------+---+------------+--------------------+-------+----------+--------------------+--------------------+
|        To the moon!|  484|mqqcbt|         r/btc|https://i.redd.it...|            131|484|        0.93|                 NaN|   moon|2021-04-14|[{sentiment, 0, 0...|               [NaN]|
|/r/bitcoin is in ...|  486|7pvojj|         r/btc|https://www.reddi...|            168|486|        0.84|Technological inf...|   moon|2018-01-12|[{sentiment, 0, 0...|[Technological, i...|
|/r/bitcoin for ye...|  470|7lcafe|         r/btc|https://www.red

In [84]:
# Clean up Combined NLP Data & Write to CSV


combined_nlp = posts_nlp.withColumn("finished_clean_lemma",
   concat_ws(" ",col("finished_clean_lemma")))

#combined_nlp = posts_nlp.withColumn("sentimentScore", col("sentimentScore.result"))
combined_nlp = posts_nlp.withColumn("sentimentScore", combined_nlp.sentimentScore.result[0]) #accesses the string correctly

# Drop the Array Columns before writing CSV
combined_nlp
combined_nlp.show()

+--------------------+-----+------+--------------+--------------------+---------------+---+------------+--------------------+-------+----------+--------------+--------------------+
|               title|score|    id|subreddit_name|                 url|number_comments|ups|upvote_ratio|                body|keyword|  datetime|sentimentScore|finished_clean_lemma|
+--------------------+-----+------+--------------+--------------------+---------------+---+------------+--------------------+-------+----------+--------------+--------------------+
|        To the moon!|  484|mqqcbt|         r/btc|https://i.redd.it...|            131|484|        0.93|                 NaN|   moon|2021-04-14|      positive|               [NaN]|
|/r/bitcoin is in ...|  486|7pvojj|         r/btc|https://www.reddi...|            168|486|        0.84|Technological inf...|   moon|2018-01-12|      positive|[Technological, i...|
|/r/bitcoin for ye...|  470|7lcafe|         r/btc|https://www.reddi...|             72|470|    

In [85]:
combined_nlp.printSchema()

root
 |-- title: string (nullable = true)
 |-- score: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- subreddit_name: string (nullable = true)
 |-- url: string (nullable = true)
 |-- number_comments: integer (nullable = true)
 |-- ups: integer (nullable = true)
 |-- upvote_ratio: float (nullable = true)
 |-- body: string (nullable = true)
 |-- keyword: string (nullable = true)
 |-- datetime: string (nullable = true)
 |-- sentimentScore: string (nullable = true)
 |-- finished_clean_lemma: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [88]:
pcc

,comment_link_id,keyword,count
0,171chv,wars,57
1,18jad6,cryptocurrency,160
2,1a4ab0,fork,287
3,1a4ab0,blocks,287
4,1a5b3b,blocksize,78
...,...,...,...
6772,qsdii9,dapp,11
6773,qss4rq,fork,80
6774,r162v,wars,25
6775,u2a1l,defi,13


In [89]:
pcu

,comment_link_id,keyword,unique_users
0,5zw489,blocks,193
1,4r2pw5,fiat,181
2,44y8ut,fiat,67
3,8bnews,currencies,150
4,6aj8gd,blocksize,44
...,...,...,...
6772,2m0jsr,decentralised,18
6773,5xxi3s,mining,15
6774,5dlx0j,energy,1
6775,8iyfn2,centralised,1


In [91]:
c.columns

Index(['title', 'score', 'id', 'subreddit_name', 'url', 'number_comments',
       'ups', 'upvote_ratio', 'body', 'keyword', 'datetime', 'sentimentScore',
       'finished_clean_lemma'],
      dtype='object')

In [129]:
unique_users = []
total_users = []

for i in range(0, len(c)):
    current_id = c.loc[i, "id"]
    current_key = c.loc[i, "keyword"]
    df_unique = pcu
    df_unique = df_unique.loc[(pcc['comment_link_id'] == current_id) & (pcc['keyword'] == current_key)]

    df_counts = pcc
    df_counts = df_counts.loc[(pcc['comment_link_id'] == current_id) & (pcc['keyword'] == current_key)]
    
    val1 = df_unique["unique_users"].values
    val1 = val1.item()
    val2 = df_counts["count"].values
    val2 = val2.item()

    unique_users.append(val1)
    total_users.append(val2)

# Add to Combined Dataframe
c["unique_commenters"] = unique_users
c["total_comments"] = total_users


ValueError: can only convert an array of size 1 to a Python scalar

In [132]:
#c = combined_nlp.toPandas()
c.to_csv("CombinedRedditData.csv", index=False)

In [133]:
c

,title,score,id,subreddit_name,url,number_comments,ups,upvote_ratio,body,keyword,datetime,sentimentScore,finished_clean_lemma,unique_commenters,total_comments
0,To the moon!,484,mqqcbt,r/btc,https://i.redd.it/mpgx359u25t61.jpg,131,484,0.93,NaN,moon,2021-04-14,positive,[NaN],[14],[126]
1,/r/bitcoin is in uproar about Coinbase not imp...,486,7pvojj,r/btc,https://www.reddit.com/r/btc/comments/7pvojj/r...,168,486,0.84,Technological inferiority when bitcoin grinds ...,moon,2018-01-12,positive,"[Technological, inferiority, bitcoin, grind, c...",[662],[170]
2,"/r/bitcoin for years: No altcoin discussion, h...",470,7lcafe,r/btc,https://www.reddit.com/r/btc/comments/7lcafe/r...,72,470,0.85,The Bitcoin subreddit reaches new levels of hy...,moon,2017-12-21,negative,"[Bitcoin, subreddit, reach, new, level, hypocr...",[31],[74]
3,This Excitement and Energy is what Bitcoin fel...,403,8elr2k,r/btc,https://www.reddit.com/r/btc/comments/8elr2k/t...,215,403,0.78,"For anyone that wasn't around in 2013, THIS is...",moon,2018-04-24,positive,"[anyone, wasnt, around, feel, like, bitcoin, R...",[423],[218]
4,MOON!,300,bqelqr,r/btc,https://i.redd.it/z7d7gbg2r4z21.jpg,69,300,0.89,NaN,moon,2019-05-19,positive,[NaN],[76],[70]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6942,John Carvalho and Roger Ver on Bitcoin Cash vs...,376,7fvpag,r/Bitcoin,https://www.youtube.com/watch?v=OJT2CbfHTpo&fe...,199,376,0.86,NaN,cash,2017-11-27,positive,[NaN],[232],[207]
6943,Exposed: How Bankers are trying to centralize ...,378,74wejs,r/Bitcoin,https://www.reddit.com/r/Bitcoin/comments/74we...,87,378,0.81,^^*Open ^^Bazaar ^^was ^^crossed-out ^^after ^...,cash,2017-10-07,positive,"[Open, Bazaar, crossedout, SX, support, retrac...",[49],[85]
6944,Donald Trump has been bashing Bitcoin becuase ...,379,pg87gt,r/Bitcoin,https://www.reddit.com/r/Bitcoin/comments/pg87...,261,379,0.77,Trump scared. Bitcoin will replace real estate...,cash,2021-09-01,positive,"[Trump, scared, Bitcoin, replace, real, estate...",[178],[257]
6945,We are developing an all in one payment card t...,376,1ssrfx,r/Bitcoin,https://www.reddit.com/r/Bitcoin/comments/1ssr...,184,376,0.94,Just like the title says we are developing a d...,cash,2013-12-13,positive,"[like, title, say, develop, dynamic, payment, ...",[6],[184]
